In [1]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

list=[[1, 2019, 1, 4],
      [1, 2019, 2, 6],
      [1, 2019, 3, 4],
      [1, 2019, 4, 2],
      [2, 2019, 1, 2],
      [2, 2019, 5, 3],
      [3, 2019, 1, 9]]


df=spark.createDataFrame(list,['Customer_ID', 'Purchase_Year', 'Purchase_Month', 'Purchases'])
df.show()

+-----------+-------------+--------------+---------+
Customer_ID|Purchase_Year|Purchase_Month|Purchases|
+-----------+-------------+--------------+---------+
 1| 2019| 1| 4|
 1| 2019| 2| 6|
 1| 2019| 3| 4|
 1| 2019| 4| 2|
 2| 2019| 1| 2|
 2| 2019| 5| 3|
 3| 2019| 1| 9|
+-----------+-------------+--------------+---------+

In [2]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
days= lambda i: i * 86400
w=Window().partitionBy("Customer_ID").orderBy("sec").rangeBetween(-days(89),0)
df.withColumn("sec", F.to_timestamp(F.concat("Purchase_Year","Purchase_Month"),"yyyyM").cast("long"))\
  .withColumn("L3", F.sum("Purchases").over(w)).orderBy("Customer_ID","Purchase_Month").drop("sec").show()

+-----------+-------------+--------------+---------+---+
Customer_ID|Purchase_Year|Purchase_Month|Purchases| L3|
+-----------+-------------+--------------+---------+---+
 1| 2019| 1| 4| 4|
 1| 2019| 2| 6| 10|
 1| 2019| 3| 4| 14|
 1| 2019| 4| 2| 12|
 2| 2019| 1| 2| 2|
 2| 2019| 5| 3| 3|
 3| 2019| 1| 9| 9|
+-----------+-------------+--------------+---------+---+

In [3]:
Customer_ID, Purchase_Year, Purchase_Month, Purchases, L3M
   1             2019             1              4     4
   1             2019             2              6     10
   1             2019             3              4     14
   1             2019             4              2     12
   2             2019             1              2     2
   2             2019             5              3     3
   3             2019             1              9     9

In [4]:
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [5]:
cschema = StructType([StructField("items",ArrayType(StructType([StructField("name", StringType()),
                      StructField("quantity", StringType())])))])


In [6]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
cschema = StructType([StructField("items",ArrayType(StructType([StructField("name", StringType()),
                      StructField("quantity", StringType())])))])
list=[[[['A','1'], ['B', '1'], ['C', '2']]]]
df=spark.createDataFrame(list,cschema)
df.show(truncate=False)
df.printSchema()
df.coalesce(1).rdd.getNumPartitions()

+------------------------+
items |
+------------------------+
[[A, 1], [B, 1], [C, 2]]|
+------------------------+

root
-- items: array (nullable = true)
 |-- element: struct (containsNull = true)
 | |-- name: string (nullable = true)
 | |-- quantity: string (nullable = true)

Out[16]: 1

In [7]:
df.withColumn("items", F.regexp_replace(F.expr("""transform(items, x-> concat_ws(', ',x.name,x.quantity))""").cast("string"),"\[|]","")).show()

+----------------+
 items|
+----------------+
A, 1, B, 1, C, 2|
+----------------+

In [8]:
df.withColumn("item2", F.array_join(F.expr("""transform(items, x-> concat_ws(',',x.name,x.quantity))"""),', ')).show()

+--------------------+-------------+
 items| item2|
+--------------------+-------------+
[[A, 1], [B, 1], ...|A,1, B,1, C,2|
+--------------------+-------------+

In [9]:
df.withColumn("items", F.expr("array_join(transform(items, \
                                i -> concat_ws(',', i.name, i.quantity)), ',')")).show()

+-----------+
 items|
+-----------+
A,1,B,1,C,2|
+-----------+

In [10]:
list=[['a','s3://path1/path6/yyy.json.gz'],
      ['b','s3://path3/path7/xxx.json.gz'],
      ['c','s3://path3/path8/aaa.json.gz'],
      ['c','s3://path4/path9/bbb.json.gz']]
df=spark.createDataFrame(list,['id','S3location'])
df.show()

+---+--------------------+
 id| S3location|
+---+--------------------+
 a|s3://path1/path6/...|
 b|s3://path3/path7/...|
 c|s3://path3/path8/...|
 c|s3://path4/path9/...|
+---+--------------------+

In [11]:
paths=[]
for i in df.collect():
  paths.append(i[1])
paths

Out[42]: ['s3://path1/path6/yyy.json.gz',
 's3://path3/path7/xxx.json.gz',
 's3://path3/path8/aaa.json.gz',
 's3://path4/path9/bbb.json.gz']

In [12]:
list=[['1,val1, val4'],
        ['2,val1'],
        ['3,val1, val2, val3'],
        ['4,val1, val2, val3, val4']]

df=spark.createDataFrame(list,['col1'])
df.show(truncate=False)

+------------------------+
col1 |
+------------------------+
1,val1, val4 |
2,val1 |
3,val1, val2, val3 |
4,val1, val2, val3, val4|
+------------------------+

In [13]:
df.withColumn("col3", F.split("col1",',')).withColumn("col1", F.element_at("col3",1)).withColumn("col2", F.expr("""filter(col3, x-> x!=col1)""")).drop("col3").show(truncate=False)

+----+---------------------------+
col1|col2 |
+----+---------------------------+
1 |[val1, val4] |
2 |[val1] |
3 |[val1, val2, val3] |
4 |[val1, val2, val3, val4]|
+----+---------------------------+

In [14]:
from pyspark.sql.types import *

In [15]:
df.withColumn("col3", F.split("col1",',')).withColumn("col1", F.element_at("col3",1)).withColumn("col2", F.array_except("col3",(F.array(F.col("col1"))))).drop("col3").printSchema()

root
-- col1: string (nullable = true)
-- col2: array (nullable = true)
 |-- element: string (containsNull = true)

In [16]:
+----+------------------------+
|col1|col2                    |
+----+------------------------+
|   1|[val1, val4]            |
|   2|[val1]                  |
|   3|[val1, val2, val3]      |
|   4|[val1, val2, val3, val4]|
+----+------------------------+

In [17]:
df.withColumn("col3", F.regexp_extract("col1",r'(\d+)',1)).show(truncate=False)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-1440734316831576> in <module> 
 ----> 1 df . withColumn ( "col3" , F . regexp_extract ( "col1" , r'(\d+)' , 2 ) ) . show ( truncate = False ) 

 /databricks/spark/python/pyspark/sql/dataframe.py in show (self, n, truncate, vertical) 
 384 print ( self . _jdf . showString ( n , 20 , vertical ) ) 
 385 else : 
 --> 386 print ( self . _jdf . showString ( n , int ( truncate ) , vertical ) ) 
 387 
 388 def __repr__ ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o3215.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 112.0 failed 1 times, most recent failure: Lost task 2.0 in stage 112.0 (TID 308, localhost, executor driver): java.lang.IndexOutOfBoundsException: No group 2
	at java.util.regex.Matcher.group(Matcher.java:538)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.project_doConsume_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at org.apache.spark.sql.execution.collect.UnsafeRowBatchUtils$.encodeUnsafeRows(UnsafeRowBatchUtils.scala:62)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:159)
	at org.apache.spark.sql.execution.collect.Collector$$anonfun$2.apply(Collector.scala:158)
	at org.apache.spark.scheduler.ResultTask.runTask(ResultTask.scala:90)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:140)
	at org.apache.spark.scheduler.Task.run(Task.scala:113)
	at org.apache.spark.executor.Executor$TaskRunner$$anonfun$13.apply(Executor.scala:533)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1541)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:539)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)

Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.org$apache$spark$scheduler$DAGScheduler$$failJobAndIndependentStages(DAGScheduler.scala:2362)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2350)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$abortStage$1.apply(DAGScheduler.scala:2349)
	at scala.collection.mutable.ResizableArray$class.foreach(ResizableArray.scala:59)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:48)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2349)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$handleTaskSetFailed$1.apply(DAGScheduler.scala:1102)
	at scala.Option.foreach(Option.scala:257)


In [18]:
df.withColumn("col3", F.split("col1",',')).withColumn("col1", F.regexp_extract("col1",r'(\d+)',1)).withColumn("col2", F.expr("""filter(col3,x-> x!=col1)""")).drop("col3").show(truncate=False)

+----+---------------------------+
col1|col2 |
+----+---------------------------+
1 |[val1, val4] |
2 |[val1] |
3 |[val1, val2, val3] |
4 |[val1, val2, val3, val4]|
+----+---------------------------+

In [19]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [20]:
list=[[1, '20180101', 'A', 1],
       [1, '20180102', 'A', 2],
       [1, '20180103', 'B', 1],
       [1, '20180104', 'A', 1],
       [1, '20180105', 'A',1],
       [1, '20180106', 'A',1],
       [2, '20180101', 'C', 1],
       [2, '20180102', 'D', 1],
       [2, '20180103', 'D', 2],
       [2, '20180103', 'D', 3]]
df= spark.createDataFrame(list,['id', 'date', 'item', 'streak'])
df=df.drop("streak")
df.show()

+---+--------+----+
 id| date|item|
+---+--------+----+
 1|20180101| A|
 1|20180102| A|
 1|20180103| B|
 1|20180104| A|
 1|20180105| A|
 1|20180106| A|
 2|20180101| C|
 2|20180102| D|
 2|20180103| D|
 2|20180103| D|
+---+--------+----+

In [21]:
w=Window().partitionBy("id","item").orderBy("date")
df.withColumn("rownum",F.row_number().over(w)).withColumn("lag", F.lag("date").over(w))\
.withColumn("lag2", F.lag("rownum").over(w))\
.withColumn("almost", F.when((F.col("date")-F.col("lag"))>1,F.lit(1)).otherwise(F.col("rownum")))\
.withColumn("almost2", F.lag("almost").over(w)).orderBy("id","date")\
.withColumn("almost2", F.when(F.col("almost2").isNull(),F.lit(0)).otherwise(F.col("almost2")))\
.withColumn("streak", F.when(F.col("almost")-F.col("almost2")>1,F.col("almost2")+1).otherwise(F.col("almost"))).show()

+---+--------+----+------+--------+----+------+-------+------+
 id| date|item|rownum| lag|lag2|almost|almost2|streak|
+---+--------+----+------+--------+----+------+-------+------+
 1|20180101| A| 1| null|null| 1| 0| 1|
 1|20180102| A| 2|20180101| 1| 2| 1| 2|
 1|20180103| B| 1| null|null| 1| 0| 1|
 1|20180104| A| 3|20180102| 2| 1| 2| 1|
 1|20180105| A| 4|20180104| 3| 4| 1| 2|
 1|20180106| A| 5|20180105| 4| 5| 4| 5|
 2|20180101| C| 1| null|null| 1| 0| 1|
 2|20180102| D| 1| null|null| 1| 0| 1|
 2|20180103| D| 2|20180102| 1| 2| 1| 2|
 2|20180103| D| 3|20180103| 2| 3| 2| 3|
+---+--------+----+------+--------+----+------+-------+------+

In [22]:
w=Window().partitionBy("id","item").orderBy("date")
w2=Window().partitionBy("id","item").orderBy(F.col("date").cast("long")).rangeBetween(-1,0)
                                                        
df.withColumn("d", F.count("item").over(w2))\
.withColumn("lag", F.lag("date").over(w))\
.orderBy("id","date").show()


+---+--------+----+---+--------+
 id| date|item| d| lag|
+---+--------+----+---+--------+
 1|20180101| A| 1| null|
 1|20180102| A| 2|20180101|
 1|20180103| B| 1| null|
 1|20180104| A| 1|20180102|
 1|20180105| A| 2|20180104|
 1|20180106| A| 2|20180105|
 2|20180101| C| 1| null|
 2|20180102| D| 1| null|
 2|20180103| D| 3|20180102|
 2|20180103| D| 3|20180103|
+---+--------+----+---+--------+

In [23]:
id, date, item, streak

1, 20180101, A, 1
1, 20180102, A, 2
1, 20180103, B, 1
1, 20180104, A, 1
2, 20180101, C, 1
2, 20180102, D, 1
2, 20180103, D, 2
2, 20180103, D, 3